# Batch Normalization - Time Series Data

## Giới thiệu

Batch Normalization cho dữ liệu time series tương tự như text data, với shape (batch, time_steps, features).

Batch Normalization sẽ normalize theo:
- Mean và variance được tính trên batch dimension và time dimension
- Chỉ giữ lại feature dimension

## Công thức

```
mean = mean(x, dim=(0, 1))  # Tính mean theo batch và time dims
var = var(x, dim=(0, 1))    # Tính variance theo batch và time dims
x_norm = (x - mean) / sqrt(var + eps)
output = gamma * x_norm + beta
```

Trong đó:
- gamma: learnable scale parameter (một cho mỗi feature)
- beta: learnable shift parameter (một cho mỗi feature)
- eps: small constant để tránh chia cho 0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class BatchNorm1D_TS:
    """
    Batch Normalization cho time series data (3D tensor: batch, time_steps, features)
    """
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        
        # Learnable parameters
        self.gamma = np.ones(num_features)  # Scale
        self.beta = np.zeros(num_features)  # Shift
        
        # Running statistics (for inference)
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)
        
    def forward(self, x, training=True):
        """
        x shape: (batch, time_steps, features)
        """
        if training:
            # Tính mean và var theo batch và time dimensions
            # Keep dims: features
            mean = np.mean(x, axis=(0, 1), keepdims=True)
            var = np.var(x, axis=(0, 1), keepdims=True)
            
            # Update running statistics
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * np.squeeze(mean)
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * np.squeeze(var)
        else:
            # Sử dụng running statistics trong inference
            mean = self.running_mean.reshape(1, 1, -1)
            var = self.running_var.reshape(1, 1, -1)
        
        # Normalize
        x_norm = (x - mean) / np.sqrt(var + self.eps)
        
        # Scale and shift
        gamma = self.gamma.reshape(1, 1, -1)
        beta = self.beta.reshape(1, 1, -1)
        output = gamma * x_norm + beta
        
        return output

print("BatchNorm1D_TS class đã được định nghĩa!")


## Ví dụ 1: 1 Sample, 1 Feature

Shape: (1, 10, 1) - 1 batch, 10 time steps, 1 feature


In [ ]:
# Ví dụ 1: 1 Sample, 1 Feature
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x1 = np.sin(t).reshape(1, 10, 1) * 5 + np.random.randn(1, 10, 1) * 2  # 1 batch, 10 time steps, 1 feature
print(f"Input shape: {x1.shape}")
print(f"Input mean: {np.mean(x1):.4f}")
print(f"Input std: {np.std(x1):.4f}")

# Khởi tạo BatchNorm
bn1 = BatchNorm1D_TS(num_features=1)

# Forward pass
output1 = bn1.forward(x1, training=True)
print(f"\nOutput shape: {output1.shape}")
print(f"Output mean: {np.mean(output1):.4f}")
print(f"Output std: {np.std(output1):.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(x1[0, :, 0], marker='o')
axes[0].set_title('Input Time Series (Before BN)')
axes[0].set_xlabel('Time Step')
axes[0].set_ylabel('Value')
axes[0].grid(True)

axes[1].plot(output1[0, :, 0], marker='o', color='orange')
axes[1].set_title('Output Time Series (After BN)')
axes[1].set_xlabel('Time Step')
axes[1].set_ylabel('Value')
axes[1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 2: 1 Sample, 2 Features

Shape: (1, 10, 2) - 1 batch, 10 time steps, 2 features


In [ ]:
# Ví dụ 2: 1 Sample, 2 Features
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x2 = np.zeros((1, 10, 2))
x2[0, :, 0] = np.sin(t) * 5 + np.random.randn(10) * 2
x2[0, :, 1] = np.cos(t) * 3 + np.random.randn(10) * 1.5
print(f"Input shape: {x2.shape}")
print(f"\nFeature 0 mean: {np.mean(x2[0, :, 0]):.4f}, std: {np.std(x2[0, :, 0]):.4f}")
print(f"Feature 1 mean: {np.mean(x2[0, :, 1]):.4f}, std: {np.std(x2[0, :, 1]):.4f}")

# Khởi tạo BatchNorm với 2 features
bn2 = BatchNorm1D_TS(num_features=2)

# Forward pass
output2 = bn2.forward(x2, training=True)
print(f"\nOutput shape: {output2.shape}")
print(f"\nFeature 0 - Output mean: {np.mean(output2[0, :, 0]):.4f}, std: {np.std(output2[0, :, 0]):.4f}")
print(f"Feature 1 - Output mean: {np.mean(output2[0, :, 1]):.4f}, std: {np.std(output2[0, :, 1]):.4f}")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x2[0, :, 0], marker='o')
axes[0, 0].set_title('Feature 0 Input (Before BN)')
axes[0, 0].set_xlabel('Time Step')
axes[0, 0].set_ylabel('Value')
axes[0, 0].grid(True)

axes[0, 1].plot(x2[0, :, 1], marker='o', color='green')
axes[0, 1].set_title('Feature 1 Input (Before BN)')
axes[0, 1].set_xlabel('Time Step')
axes[0, 1].set_ylabel('Value')
axes[0, 1].grid(True)

axes[1, 0].plot(output2[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Feature 0 Output (After BN)')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].set_ylabel('Value')
axes[1, 0].grid(True)

axes[1, 1].plot(output2[0, :, 1], marker='o', color='red')
axes[1, 1].set_title('Feature 1 Output (After BN)')
axes[1, 1].set_xlabel('Time Step')
axes[1, 1].set_ylabel('Value')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 3: 2 Samples, 1 Feature

Shape: (2, 10, 1) - 2 batches, 10 time steps, 1 feature

Trong trường hợp này, BatchNorm sẽ tính mean và variance trên cả 2 samples trong batch.


In [ ]:
# Ví dụ 3: 2 Samples, 1 Feature
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x3 = np.zeros((2, 10, 1))
x3[0, :, 0] = np.sin(t) * 5 + np.random.randn(10) * 2
x3[1, :, 0] = np.cos(t) * 3 + np.random.randn(10) * 1.5
print(f"Input shape: {x3.shape}")
print(f"\nSample 0 mean: {np.mean(x3[0]):.4f}, std: {np.std(x3[0]):.4f}")
print(f"Sample 1 mean: {np.mean(x3[1]):.4f}, std: {np.std(x3[1]):.4f}")
print(f"Overall mean (across batch): {np.mean(x3):.4f}, std: {np.std(x3):.4f}")

# Khởi tạo BatchNorm
bn3 = BatchNorm1D_TS(num_features=1)

# Forward pass
output3 = bn3.forward(x3, training=True)
print(f"\nOutput shape: {output3.shape}")
print(f"\nSample 0 - Output mean: {np.mean(output3[0]):.4f}, std: {np.std(output3[0]):.4f}")
print(f"Sample 1 - Output mean: {np.mean(output3[1]):.4f}, std: {np.std(output3[1]):.4f}")
print(f"Overall output mean: {np.mean(output3):.4f}, std: {np.std(output3):.4f}")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(x3[0, :, 0], marker='o')
axes[0, 0].set_title('Sample 0 Input (Before BN)')
axes[0, 0].set_xlabel('Time Step')
axes[0, 0].set_ylabel('Value')
axes[0, 0].grid(True)

axes[0, 1].plot(x3[1, :, 0], marker='o', color='green')
axes[0, 1].set_title('Sample 1 Input (Before BN)')
axes[0, 1].set_xlabel('Time Step')
axes[0, 1].set_ylabel('Value')
axes[0, 1].grid(True)

axes[1, 0].plot(output3[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Sample 0 Output (After BN)')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].set_ylabel('Value')
axes[1, 0].grid(True)

axes[1, 1].plot(output3[1, :, 0], marker='o', color='red')
axes[1, 1].set_title('Sample 1 Output (After BN)')
axes[1, 1].set_xlabel('Time Step')
axes[1, 1].set_ylabel('Value')
axes[1, 1].grid(True)
plt.tight_layout()
plt.show()


## Ví dụ 4: 2 Samples, 2 Features

Shape: (2, 10, 2) - 2 batches, 10 time steps, 2 features

Đây là trường hợp đầy đủ nhất với nhiều samples và nhiều features.


In [ ]:
# Ví dụ 4: 2 Samples, 2 Features
np.random.seed(42)
t = np.linspace(0, 4*np.pi, 10)
x4 = np.zeros((2, 10, 2))
x4[0, :, 0] = np.sin(t) * 5 + np.random.randn(10) * 2
x4[0, :, 1] = np.cos(t) * 3 + np.random.randn(10) * 1.5
x4[1, :, 0] = np.sin(2*t) * 4 + np.random.randn(10) * 1.8
x4[1, :, 1] = np.cos(2*t) * 2 + np.random.randn(10) * 1.2
print(f"Input shape: {x4.shape}")

for b in range(2):
    for f in range(2):
        print(f"Sample {b}, Feature {f} - mean: {np.mean(x4[b, :, f]):.4f}, std: {np.std(x4[b, :, f]):.4f}")

# Khởi tạo BatchNorm với 2 features
bn4 = BatchNorm1D_TS(num_features=2)

# Forward pass
output4 = bn4.forward(x4, training=True)
print(f"\nOutput shape: {output4.shape}")

for b in range(2):
    for f in range(2):
        print(f"Sample {b}, Feature {f} - Output mean: {np.mean(output4[b, :, f]):.4f}, std: {np.std(output4[b, :, f]):.4f}")

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for b in range(2):
    for f in range(2):
        axes[b, f*2].plot(x4[b, :, f], marker='o')
        axes[b, f*2].set_title(f'Sample {b}, Feat {f} Input')
        axes[b, f*2].set_xlabel('Time Step')
        axes[b, f*2].set_ylabel('Value')
        axes[b, f*2].grid(True)
        
        axes[b, f*2+1].plot(output4[b, :, f], marker='o', color='orange')
        axes[b, f*2+1].set_title(f'Sample {b}, Feat {f} Output')
        axes[b, f*2+1].set_xlabel('Time Step')
        axes[b, f*2+1].set_ylabel('Value')
        axes[b, f*2+1].grid(True)

plt.tight_layout()
plt.show()


## Ví dụ 5: Shape phức tạp - (batch_size, time, V*len(lag)+number_of_time)

Ví dụ với:
- batch_size = 2
- time = 10
- V = 2 (số biến)
- len(lag) = 3 (lag window)
- number_of_time = 2 (features thời gian bổ sung)
- Total features = V*len(lag) + number_of_time = 2*3 + 2 = 8

Shape: (2, 10, 8)


In [ ]:
# Ví dụ 5: Shape phức tạp (batch_size, time, V*len(lag)+number_of_time)
np.random.seed(42)

# Tham số
batch_size = 2
time = 10
V = 2  # số biến
len_lag = 3  # độ dài lag window
number_of_time = 2  # features thời gian bổ sung
num_features = V * len_lag + number_of_time  # 2*3 + 2 = 8

print(f"Tham số:")
print(f"  batch_size = {batch_size}")
print(f"  time = {time}")
print(f"  V = {V} (số biến)")
print(f"  len(lag) = {len_lag} (lag window)")
print(f"  number_of_time = {number_of_time}")
print(f"  Total features = V*len(lag) + number_of_time = {V}*{len_lag} + {number_of_time} = {num_features}")

# Tạo dữ liệu mô phỏng
t = np.linspace(0, 4*np.pi, time)
x5 = np.zeros((batch_size, time, num_features))

for b in range(batch_size):
    # Features từ lag windows (V*len_lag features)
    for v in range(V):
        base_signal = np.sin((v+1)*t) * (5 + b*2) + np.random.randn(time) * 2
        for lag in range(len_lag):
            feature_idx = v * len_lag + lag
            x5[b, :, feature_idx] = np.roll(base_signal, lag) + np.random.randn(time) * 0.5
    
    # Features thời gian bổ sung (number_of_time features)
    x5[b, :, V*len_lag] = np.cos(t) * 3 + np.random.randn(time) * 1.5
    x5[b, :, V*len_lag+1] = t / np.max(t) * 10 + np.random.randn(time) * 1

print(f"\nInput shape: {x5.shape}")

# Khởi tạo BatchNorm với số features phức tạp
bn5 = BatchNorm1D_TS(num_features=num_features)
output5 = bn5.forward(x5, training=True)

print(f"\nOutput shape: {output5.shape}")

# Hiển thị statistics cho một vài features
print(f"\nSample 0 - Feature 0 (V0, lag0): Input mean={np.mean(x5[0, :, 0]):.4f}, Output mean={np.mean(output5[0, :, 0]):.4f}")
print(f"Sample 0 - Feature 3 (V1, lag0): Input mean={np.mean(x5[0, :, 3]):.4f}, Output mean={np.mean(output5[0, :, 3]):.4f}")
print(f"Sample 0 - Feature 6 (time feature 0): Input mean={np.mean(x5[0, :, 6]):.4f}, Output mean={np.mean(output5[0, :, 6]):.4f}")

# Visualize một vài features
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes[0, 0].plot(x5[0, :, 0], marker='o')
axes[0, 0].set_title('Sample 0, Feature 0 (V0, lag0) Input')
axes[0, 0].set_xlabel('Time Step')
axes[0, 0].grid(True)
axes[0, 1].plot(x5[0, :, 3], marker='o', color='green')
axes[0, 1].set_title('Sample 0, Feature 3 (V1, lag0) Input')
axes[0, 1].set_xlabel('Time Step')
axes[0, 1].grid(True)
axes[0, 2].plot(x5[0, :, 6], marker='o', color='blue')
axes[0, 2].set_title('Sample 0, Feature 6 (time) Input')
axes[0, 2].set_xlabel('Time Step')
axes[0, 2].grid(True)

axes[1, 0].plot(output5[0, :, 0], marker='o', color='orange')
axes[1, 0].set_title('Sample 0, Feature 0 Output')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].grid(True)
axes[1, 1].plot(output5[0, :, 3], marker='o', color='red')
axes[1, 1].set_title('Sample 0, Feature 3 Output')
axes[1, 1].set_xlabel('Time Step')
axes[1, 1].grid(True)
axes[1, 2].plot(output5[0, :, 6], marker='o', color='purple')
axes[1, 2].set_title('Sample 0, Feature 6 Output')
axes[1, 2].set_xlabel('Time Step')
axes[1, 2].grid(True)

plt.tight_layout()
plt.show()
